<a href="https://colab.research.google.com/github/mtuan8820/learnNLP/blob/main/article_spinner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv


--2024-09-01 08:59:25--  https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 172.67.213.166, 104.21.23.210, 2606:4700:3031::6815:17d2, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|172.67.213.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5085081 (4.8M) [text/csv]
Saving to: ‘bbc_text_cls.csv’

bbc_text_cls.csv    100%[===================>]   4.85M  --.-KB/s    in 0.03s   

2024-09-01 08:59:25 (182 MB/s) - ‘bbc_text_cls.csv’ saved [5085081/5085081]



In [1]:
import numpy as np
import pandas as pd
import textwrap
import nltk
from nltk import word_tokenize
from nltk.tokenize.treebank  import TreebankWordDetokenizer


In [3]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
df = pd.read_csv('bbc_text_cls.csv')

In [6]:
labels = set(df['labels'])
labels

{'business', 'entertainment', 'politics', 'sport', 'tech'}

In [8]:
label = 'tech'

In [10]:
texts = df[df.labels == label].text
texts.head()

,text
1824,Ink helps drive democracy in Asia\n\nThe Kyrgy...
1825,China net cafe culture crackdown\n\nChinese au...
1826,Microsoft seeking spyware trojan\n\nMicrosoft ...
1827,Digital guru floats sub-$100 PC\n\nNicholas Ne...
1828,Technology gets the creative bug\n\nThe hi-tec...


In [12]:
prob = {}

for doc in texts:
  lines = doc.split("\n")
  for line in lines:
    tokens = word_tokenize(line)
    for i in range(len(tokens)-2):
      t_0 = tokens[i]
      t_1 = tokens[i+1]
      t_2 = tokens[i+2]
      key = (t_0, t_2)
      if key not in prob:
        prob[key] = {}
      # add count for middle word
      if t_1 not in prob[key]:
        prob[key][t_1] = 1
      else:
        prob[key][t_1] += 1

In [13]:
# normalize probabilities
for key, d in prob.items():
  total = sum(d.values())
  for k,v in d.items():
    d[k] /= v/total

In [14]:
detokenizer = TreebankWordDetokenizer()


In [15]:
def sample_word(d):
  p0 = np.random.random()
  cumulative = 0
  for t, p in d.items():
    cumulative += p
    if p0 < cumulative:
      return t
  assert(False) # should not reach


In [22]:
def spin_line(line):
  tokens = word_tokenize(line)
  output = [tokens[0]]
  i = 0
  while i < len(tokens)-2:
    t_0 = tokens[i]
    t_1 = tokens[i+1]
    t_2 = tokens[i+2]
    key = (t_0, t_2)
    p_dist = prob[key]
    if len(p_dist) > 1 and np.random.random() <0.3:
      #  replace middle word
      # 0.3: way to control how often words are replaced
      middle = sample_word(p_dist)
      output.append(t_1)
      output.append('<'+middle+'>')
      output.append(t_2)
      i+=2
    else:
      output.append(t_1)
      i+=1
  if i == len(tokens)-2:
    output.append(tokens[-1])
  return detokenizer.detokenize(output)

In [23]:
def spin_doc(doc):
  lines = doc.split("\n")
  new_lines = []
  for line in lines:
    if line:
      new_line = spin_line(line)
    else:
      new_line = line
    new_lines.append(new_line)
  return "\n".join(new_lines)

In [24]:
np.random.seed(1234)


In [25]:
i = np.random.choice(texts.shape[0])
doc = texts.iloc[i]
new_doc = spin_doc(doc)

In [26]:
print(textwrap.fill(
    new_doc, replace_whitespace=False, fix_sentence_endings=True))

Format wars could 'confuse users'

Technology firms Sony, Philips,
Matsushita and Samsung are developing a common <legal> way to stop
people pirating <with> digital music <camera> and video.

The firms
want to make <create> a system that ensures files play on <against>
the hardware <programmes> they make but also thwarts illegal copying .
The move could mean <be> more confusion for consumers already faced by
many different, and conflicting, content control systems <system>,
experts warned . They say there are no guarantees <guarantees> the
system will even prevent piracy.

Currently many online stores wrap up
downloadable files in an own-brand control system that means they can
<typically> only be played on a <such> small number <population> of
media players . Systems <But> that limit <controls> what people can do
<,> with the files they download are known <classed> as Digital Rights
Management systems . By setting up the alliance to work on <towards> a
common <unique> control system, 